Following Longxu's advice, we will deliberately break the sentences and see if the training differs

In [ ]:
import os
import glob
import json
import tqdm

In [ ]:
json_path = '/home/aiops/zhuty/ret_pretraining_data/c4_news_id_added/train/'
def read_jsonl(paths):
    """Read a JSON Lines file and return a list of documents."""
    documents = []
    for file_path in tqdm.tqdm(paths):
        with open(file_path, 'r') as file:
            for line in file:
                document = json.loads(line)
                document['text'] = document['contents']
                del document['contents']
                documents.append(document)
    return documents


In [ ]:
documents = read_jsonl(glob.glob(os.path.join(json_path, '*.jsonl'))[:])
len(documents)
num_of_paragraphs = []
all_paragraphs = []
for doc in documents:
    paragraphs = doc['text'].split("\n")
    num_of_paragraphs.append(len(paragraphs))
    all_paragraphs.extend(paragraphs)

# get some stats on the number of paragraphs
print("Total number of paragraphs: ", sum(num_of_paragraphs))
print("Average number of paragraphs: ", sum(num_of_paragraphs)/len(num_of_paragraphs))
print("Max number of paragraphs: ", max(num_of_paragraphs))
print("Min number of paragraphs: ", min(num_of_paragraphs))

import matplotlib.pyplot as plt
# plot histogram of the number of paragraphs
plt.hist(num_of_paragraphs, bins=100)
plt.show()




In [ ]:
import random
random.seed(42)
random.shuffle(all_paragraphs)

In [ ]:
def split_paragraphs(paragraphs, paragraph_counts):
    documents = []
    start = 0

    for count in tqdm.tqdm(paragraph_counts):
        end = start + count
        documents.append(paragraphs[start:end])
        start = end

    return documents

documents = split_paragraphs(all_paragraphs, num_of_paragraphs)

In [ ]:
assert len(all_paragraphs) == sum(num_of_paragraphs)

In [ ]:
len(all_paragraphs)

In [ ]:
sum(num_of_paragraphs)

In [ ]:
new_docs = []
for i ,doc in enumerate(documents):
    new_docs.append({
        'text': '\n'.join(doc),
        'id': "adv_" + str(i),
    })

In [ ]:
def write_jsonl(dir_path, documents, chunk_size=100000):
    """Write a list of documents to chunks of JSON Lines file."""
    num_chunks = len(documents) // chunk_size + 1
    for i in range(num_chunks):
        chunk_path = os.path.join(dir_path, f'chunk_{i}.jsonl')
        start = i * chunk_size
        end = min((i + 1) * chunk_size, len(documents))
        with open(chunk_path, 'w') as file:
            for document in documents[start:end]:
                file.write(json.dumps(document) + '\n')


In [ ]:
dir_path = '/home/aiops/zhuty/ret_pretraining_data/c4_news_adv/train'
write_jsonl(dir_path, new_docs)

In [ ]:
print(len(new_docs))

Explore how to process zst file

In [2]:
import os
import zstandard as zstd
import glob
import tqdm

def decompress_zst_file(zst_file, output_dir):
    with open(zst_file, 'rb') as compressed:
        decomp = zstd.ZstdDecompressor()
        with decomp.stream_reader(compressed) as reader:
            jsonl_file = zst_file.replace('.zst', '')
            output_path = os.path.join(output_dir, os.path.basename(jsonl_file))
            with open(output_path, 'wb') as destination:
                while True:
                    chunk = reader.read(16384)  # Read in chunks of 16 KB
                    if not chunk:
                        break
                    destination.write(chunk)

def decompress_all_files(file_list, output_dir):
    for file in tqdm.tqdm(file_list):
        decompress_zst_file(file, output_dir)

# Example usage
for split in ['train', 'validation']:
    file_list = glob.glob(os.path.join('/home/aiops/zhuty/ret_pretraining_data/cc_raw', split, "RedPajamaCommonCrawl", '*.zst'))
    print(len(file_list))
    output_dir = os.path.join('/home/aiops/zhuty/ret_pretraining_data/cc', split)
    decompress_all_files(file_list, output_dir)


100


100%|██████████| 100/100 [05:56<00:00,  3.57s/it]


629


100%|██████████| 629/629 [00:15<00:00, 39.37it/s]


In [ ]:
import json
from lingua import Language, LanguageDetectorBuilder
import os
import zstandard as zstd
import glob
import tqdm

def read_jsonl(paths):
    """Read a JSON Lines file and return a list of documents."""
    documents = []
    for file_path in tqdm.tqdm(paths, disable=True):
        with open(file_path, 'r') as file:
            for line in file:
                document = json.loads(line)
                documents.append(document)
    return documents


def write_jsonl(file_name, documents):
    """Write a list of documents to a JSON Lines file."""
    with open(file_name, 'w') as file:
        for document in documents:
            file.write(json.dumps(document) + '\n')


for split in ['train', 'test', 'validation']:
    chunk_paths = glob.glob(os.path.join('/home/aiops/zhuty/ret_pretraining_data/rpwiki', split, '*.jsonl'))
    for chunk_num, chunk_path in tqdm.tqdm(enumerate(chunk_paths)):
        documents = read_jsonl([chunk_path])
        detector = LanguageDetectorBuilder.from_all_languages().build()
        languages = detector.detect_languages_in_parallel_of([doc['text'] for doc in documents])
        print(len(languages))
        assert len(documents) == len(languages)
        english_documents = [doc for doc, lang in zip(documents, languages) if lang == Language.ENGLISH]
        print(len(english_documents), "percentage: ", len(english_documents)/len(documents))
        output_file =  os.path.join('/home/aiops/zhuty/ret_pretraining_data/rpwiki_en', split, f'chunk_{chunk_num}.jsonl')
        write_jsonl(output_file, english_documents)

In [ ]:
detector = LanguageDetectorBuilder.from_all_languages().build()

In [ ]:
languages = detector.detect_languages_in_parallel_of([doc['text'] for doc in documents])

In [ ]:
len(languages)

In [ ]:
# get english documents
english_documents = [doc for doc, lang in zip(documents, languages) if lang == Language.ENGLISH]
print(len(english_documents))

In [ ]:
english_documents[0]